### 모델 로드

### 로드할 모델들
- onehotencode 모델 : 사용자 입력 받은 category 데이터 변환
- sampling 모델 : 사용자 입력 받은 continue 데이터 변환 (onehotencode 결과 포함)
- predict 모델 : 변환한 데이터 입력으로 label 예측값 리턴
- labelencode 모델 : predict한 데이터를 원래 label 형태로 변환 후 리턴 (encode 자체를 로드 할 필요보다는 매핑 순서만 받아도 될 듯?)

In [1]:
import pickle 
load_file_name = f'../../models/titanic_disaster.pkl'

# 모델과 선택된 특성을 로드
with open(load_file_name, 'rb') as load_file:
    model_info = pickle.load(load_file)

onehotencoders = model_info['onehotencoders']
standardscaler = model_info['standardscaler']
logisticregression = model_info['logisticregression']
labelencoder = model_info['labelencoder']

print("모델과 선택된 특성이 로드되었습니다.")

모델과 선택된 특성이 로드되었습니다.


In [2]:
len(onehotencoders)

5

In [3]:
type(onehotencoders[0]), type(standardscaler), type(logisticregression), type(labelencoder), 

(sklearn.preprocessing._encoders.OneHotEncoder,
 sklearn.preprocessing._data.StandardScaler,
 sklearn.linear_model._logistic.LogisticRegression,
 sklearn.preprocessing._label.LabelEncoder)

In [4]:
for model in onehotencoders:
    print(model.categories_)

[array([1, 2, 3])]
[array(['female', 'male'], dtype=object)]
[array([0, 1, 2, 3, 4, 5, 8])]
[array([0, 1, 2, 3, 4, 5, 6])]
[array(['C', 'Q', 'S'], dtype=object)]


### 모델 테스트 

##### 테스트 입력 셋
- Pclass	Sex	SibSp	Parch	Embarked	Age	Fare	Survived
- 3	male	1	0	S	22.0	7.2500	0
- 1	female	1	0	C	38.0	71.2833	1
- 3	female	0	0	S	26.0	7.9250	1
- 1	female	1	0	S	35.0	53.1000	1
- 3	male	0	0	S	35.0	8.0500	0

In [5]:
import pandas as pd

In [6]:
def transform_with_onehot(input_values, onehot_encoders):
    """
    입력값을 순서대로 원핫인코딩 변환
    
    Parameters:
    input_values : list - 변환할 입력값들 [Pclass, Sex, SibSp, Parch, Embarked]
    onehot_encoders : list - 순서대로 매칭되는 원핫인코더 모델들
    
    Returns:
    DataFrame - 원핫인코딩된 결과를 담은 데이터프레임
    """
    # 컬럼명들 (원핫인코딩 시 사용했던 순서대로)
    columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
    
    # 입력값을 DataFrame으로 변환
    input_df = pd.DataFrame([input_values], columns=columns)
    
    # 결과를 저장할 빈 데이터프레임
    result_df = pd.DataFrame()
    
    for col, encoder in zip(columns, onehot_encoders):
        # DataFrame의 컬럼을 사용하여 변환
        encoded = encoder.transform(input_df[[col]]).toarray()
        # 특성 이름 가져오기
        feature_names = encoder.get_feature_names_out(input_features=[col])
        # 결과를 DataFrame으로 변환하여 추가
        encoded_df = pd.DataFrame(
            data=encoded, 
            columns=feature_names, 
            index=input_df.index
        )
        # 결과 데이터프레임에 추가
        result_df = pd.concat([result_df, encoded_df], axis=1)
    
    return result_df

In [7]:
input_values = [3, 'male', 1, 0, 'S']
encoded_df = transform_with_onehot(input_values, onehotencoders)
encoded_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
def transform_with_sampling(encoded_df, continuous_columns, continuous_values, standard_scaler):
    """
    원핫인코딩된 데이터프레임에 연속형 변수를 추가하고 전체를 스케일링
    
    Parameters:
    encoded_df : DataFrame - 원핫인코딩된 데이터프레임
    continuous_columns : list - 연속형 변수 컬럼명들
    continuous_values : list - 연속형 변수 값들
    standard_scaler : StandardScaler - 스케일링 모델
    
    Returns:
    DataFrame - 최종 변환된 데이터프레임
    """
    # 연속형 변수를 DataFrame으로 만들어서 원핫인코딩 결과와 결합
    continuous_df = pd.DataFrame(
        data=[continuous_values],
        columns=continuous_columns,
        index=encoded_df.index
    )
    
    # 전체 데이터 결합
    combined_df = pd.concat([encoded_df, continuous_df], axis=1)

    # 전체 데이터에 대해 스케일링 적용
    scaled_data = standard_scaler.transform(combined_df)
    
    # 스케일링된 결과를 DataFrame으로 변환
    scaled_df = pd.DataFrame(
        data=scaled_data,
        columns=combined_df.columns,
        index=combined_df.index
    )
    
    return scaled_df

In [9]:
continuous_columns = ['Fare', 'Age']
continuous_values = [7.2500, 22.0]
sampling_df = transform_with_sampling(encoded_df, continuous_columns, continuous_values, standardscaler)
sampling_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S,Fare,Age
0,-0.565685,-0.510152,0.902587,-0.737695,0.737695,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,...,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352,-0.482043,-0.307562,0.615838,-0.502445,-0.559667


In [11]:
# 각 레코드를 처리하는 함수
def process_record(record):
    # 카테고리형 변수 (OneHotEncoder 용)
    input_values = record[:5]  # Pclass, Sex, SibSp, Parch, Embarked
    
    # 연속형 변수
    continuous_columns = ['Fare', 'Age']
    continuous_values = record[5:]  # Age, Fare
    
    # OneHotEncoding 변환
    encoded_df = transform_with_onehot(input_values, onehotencoders)
    
    # Sampling 변환
    sampling_df = transform_with_sampling(encoded_df, continuous_columns, continuous_values, standardscaler)
    
    return sampling_df

In [22]:
# 데이터를 리스트로 구성
test_records = [
    # Pclass, Sex,    SibSp, Parch, Embarked,    Fare,  Age
    [3,       'male',   1,    0,     'S',       7.2500,  22.0],
    [1,       'female', 1,    0,     'C',       71.2833, 38.0],
    [3,       'female', 0,    0,     'S',       7.9250,  26.0],
    [1,       'female', 1,    0,     'S',       53.1000, 35.0],
    [3,       'male',   0,    0,     'S',       8.0500,  35.0]
]

# 모든 레코드 처리
processed_records = []
for record in test_records:
    processed_df = process_record(record)
    processed_records.append(processed_df)
    

# 예측 결과 수집
predictions = []
for df in processed_records:
    pred = logisticregression.predict(df)[0]
    probs = logisticregression.predict_proba(df)[0]
    print(f"Predicted Survival: {pred}, {probs[0]}, {probs[1]}")
    predictions.append(pred)

Predicted Survival: 0, 0.8939094689569732, 0.10609053104302683
Predicted Survival: 1, 0.056608339767074245, 0.9433916602329258
Predicted Survival: 1, 0.3587263928106228, 0.6412736071893772
Predicted Survival: 1, 0.07059513635513304, 0.929404863644867
Predicted Survival: 0, 0.9356343368239668, 0.06436566317603325


In [18]:
# 예측 결과만을 담은 새로운 데이터프레임 생성
prediction_df = pd.DataFrame({
    'Predicted_Survival': predictions
})

labelencoder.inverse_transform(predictions) 
# 나중에 string 이던 label을 transform 되돌릴 일 있을 때 사용

array([0, 1, 1, 1, 0])